# We will separate the sample into a list of triplets in **low-mass compact** systems

In [22]:
import numpy as np
import pandas as pd

In [23]:
# read list of all known planets (post processed exoplanet.eu data)
### isys      : index of system
### totpl_sys : number of planets in the system
### ipl_sys   : index of planet in the system

data_full=pd.read_csv('../planets_july2021.txt',delimiter='\s{3,}',header=None,engine='python',names=['isys','totpl_sys','ipl_sys','pl_name','m','m_est','r','P','a','e','inc','w','lambda','idetect','st_met','st_m','st_r','st_age','date'])
data_full

,isys,totpl_sys,ipl_sys,pl_name,m,m_est,r,P,a,e,inc,w,lambda,idetect,st_met,st_m,st_r,st_age,date
0,1,1,1,11 Com b,6167.6483,6167.6483,0.0000,3.260300e+02,1.2900,0.23100,0.000,94.80,0.0,1.0,-0.35,2.70,19.00,0.00,2008.0
1,2,1,1,11 Oph b,6676.3203,6676.3203,0.0000,7.300000e+05,243.0000,0.00000,0.000,0.00,0.0,2.0,0.00,0.02,0.00,0.01,2007.0
2,3,1,1,11 UMi b,3338.1601,3338.1601,0.0000,5.162200e+02,1.5400,0.08000,0.000,117.63,0.0,1.0,0.04,1.80,24.08,1.56,2009.0
3,4,1,1,14 And b,1694.5137,1694.5137,0.0000,1.858400e+02,0.8300,0.00000,0.000,0.00,0.0,1.0,-0.24,2.20,11.00,0.00,2008.0
4,5,2,1,14 Her b,1475.1489,1475.1489,0.0000,1.773400e+03,2.7700,0.36900,0.000,22.60,0.0,1.0,0.43,0.90,0.71,5.10,2002.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4712,3542,4,1,ups And b,197.1104,197.1104,0.0000,4.617100e+00,0.0590,0.01186,90.000,44.52,0.0,1.0,0.09,1.27,1.63,3.80,1996.0
4713,3542,4,2,ups And c,2893.0721,2893.0721,0.0000,2.409370e+02,0.8610,0.24450,11.347,247.63,0.0,1.0,0.09,1.27,1.63,3.80,1999.0
4714,3542,4,3,ups And d,7496.5539,7496.5539,0.0000,1.281439e+03,2.5500,0.31600,25.609,252.99,0.0,1.0,0.09,1.27,1.63,3.80,1999.0
4715,3542,4,4,ups And e,336.6773,336.6773,0.0000,3.848860e+03,5.2456,0.00536,0.000,7.30,0.0,1.0,0.09,1.27,1.63,3.80,2010.0


In [24]:
# returns planet's name when given (isys,ipl_sys)
def plname(df,isys,ipl_sys):
    plname = df[(df['isys']==isys)&(df['ipl_sys']==ipl_sys)]['pl_name'].values[0]
    return(plname)

In [25]:
# keep systems with at least 3 planets
data = data_full[data_full['totpl_sys']>2]

In [26]:
datag = data.groupby('isys') # datag can be divided into individual dataframes for each system
sysl = sorted(list(set(data['isys']))) # least of isys with at least 2 planets

####

### Create table of triplets

Only between triplets of **adjacent** planets, in systems where every planet has mass **lower** than 20 $m_\oplus$, and period ratios **between** 1.2 and 1.7. 

In [30]:
triplets=[] # main triplet table
for isys in sysl:
    i=1
    data_isys = datag.get_group(isys) # dataframe for system isys
    itotpl = data_isys['totpl_sys'].values[0] # number of planets in isys
    
    # lowmass condition
    m_l = np.array(data_isys['m'] > 20)
    if True in m_l:
        continue
    
    # loop per planetary triplet in isys
    counter=0
    for intervalo in range(itotpl-2):
        Pi0 = data_isys['P'].values[counter]
        Pi1 = data_isys['P'].values[counter+1]
        Pi2 = data_isys['P'].values[counter+2]

        mi0 = data_isys['m'].values[counter]
        mi1 = data_isys['m'].values[counter+1]
        mi2 = data_isys['m'].values[counter+2]

        id0 = data_isys['ipl_sys'].values[counter]
        id1 = data_isys['ipl_sys'].values[counter+1]
        id2 = data_isys['ipl_sys'].values[counter+2]
        
        pl1=plname(data,isys,id0)
        pl2=plname(data,isys,id1)
        pl3=plname(data,isys,id2)
        
        p1p0_ = np.round(Pi1/Pi0,decimals=7)
        p2p1_ = np.round(Pi2/Pi1,decimals=7)
        
        # compact condition
        if (p1p0_>1.2)and(p1p0_<1.7)and(p2p1_>1.2)and(p2p1_<1.7):
            compact=True
        else:
            compact=False
        
        if Pi0!=0. and Pi1!=0. and Pi2!=0. and compact==True:
            triplets.append([p1p0_,p2p1_,pl1,pl2,pl3,mi0,mi1,mi2,Pi0,Pi1,Pi2])
        
        counter+=1
        
print('# of lowmass compact triplets: '+str(len(triplets)))

# list to dataframe
triplets = pd.DataFrame(np.matrix(triplets),columns=['p1p0','p2p1','pl_1','pl_2','pl_3','m1','m2','m3','P1','P2','P3'])
# especificar dtypes
triplets = triplets.astype(dtype={'p1p0':float,'p2p1':float,'pl_1':str,'pl_2':str,'pl_3':str,'m1':float,'m2':float,'m3':float,'P1':float,'P2':float,'P3':float})

triplets.head(6) # show first 6 rows

# of lowmass compact triplets: 57


,p1p0,p2p1,pl_1,pl_2,pl_3,m1,m2,m3,P1,P2,P3
0,1.386851,1.594834,GJ 667 C c,GJ 667 C f,GJ 667 C e,3.8150,2.7023,2.7023,28.1400,39.0260,62.2400
1,1.594834,1.471883,GJ 667 C f,GJ 667 C e,GJ 667 C d,2.7023,2.7023,5.0994,39.0260,62.2400,91.6100
2,1.575758,1.514569,HD 158259 b,HD 158259 c,HD 158259 d,2.7023,5.5000,5.3093,2.1780,3.4320,5.1980
3,1.514569,1.530204,HD 158259 c,HD 158259 d,HD 158259 e,5.5000,5.3093,5.7861,3.4320,5.1980,7.9540
4,1.530204,1.512447,HD 158259 d,HD 158259 e,HD 158259 f,5.3093,5.7861,6.3902,5.1980,7.9540,12.0300
5,1.264628,1.491618,HD 215152 b,HD 215152 c,HD 215152 d,1.9965,1.5263,2.7058,5.7594,7.2835,10.8642


.
### Save files

In [31]:
triplets.to_csv('known_lowmass_compact_triplets.dat',index=False,header=None)